<a href="https://colab.research.google.com/github/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set/blob/coords5/train_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [12]:
!pip install icecream
#%tensorflow_version 1.x
import os
!pip install kora
from kora import drive
import time
!pip install colorama

ds_path='/content/drive/My Drive/PhD/datasets/cv_data/'

In [13]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
git_clone = True

if git_clone == True:
  os.chdir('/content')
  %rm -rf FCN_ConvLSTM_Crop_Recognition_Open_Set
  !git clone --branch coords5 https://github.com/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set.git

Cloning into 'FCN_ConvLSTM_Crop_Recognition_Open_Set'...
remote: Enumerating objects: 2173, done.
remote: Counting objects: 100% (2173/2173), done.
remote: Compressing objects: 100% (1604/1604), done.
remote: Total 2173 (delta 1429), reused 1059 (delta 365), pack-reused 0
Receiving objects: 100% (2173/2173), 37.49 MiB | 23.96 MiB/s, done.
Resolving deltas: 100% (1429/1429), done.


## Download images into proper folder

In [15]:
!cp -r /content/drive/MyDrive/PhD/datasets/cv_data /content/FCN_ConvLSTM_Crop_Recognition_Open_Set/dataset/dataset/

In [16]:
os.chdir('/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src')
os.getcwd()
os.listdir()

['mosaic.py.old',
 'postprocessing.py',
 'model_input_mode.py',
 'model_best_UUnet4ConvLSTM_jun_cv_criteria_0_92',
 'densnet_timedistributed.py',
 '__init__.py',
 'keras_weighted_categorical_crossentropy.py',
 'deb.py',
 'model.py',
 'modelArchitecture.py',
 'main.py',
 'patch_extractor.py',
 'parameters',
 'mosaic.py',
 'metrics.py',
 'monitor.py',
 'dataset.py',
 'generator.py',
 'open_set.py',
 'obj',
 'dataSource.py',
 'densnet.py',
 'analysis']

In [17]:
from colorama import init
init()
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, Conv2DTranspose
# from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad 
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow.keras as keras

import numpy as np
from sklearn.utils import shuffle
import cv2
import argparse
import tensorflow as tf

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import metrics
import sys
import glob

from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report
# Local
from densnet import DenseNetFCN
from densnet_timedistributed import DenseNetFCNTimeDistributed

#from metrics import fmeasure,categorical_accuracy
import deb
from keras_weighted_categorical_crossentropy import weighted_categorical_crossentropy, sparse_accuracy_ignoring_last_label, weighted_categorical_crossentropy_ignoring_last_label, categorical_focal_ignoring_last_label, weighted_categorical_focal_ignoring_last_label
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import ConvLSTM2D, UpSampling2D, multiply
from tensorflow.keras.regularizers import l1,l2
import time
import pickle
#from tensorflow.keras_self_attention import SeqSelfAttention
import pdb
import pathlib
from pathlib import Path, PureWindowsPath
from tensorflow.keras.layers import Conv3DTranspose, Conv3D

from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter


#from datagenerator import DataGenerator
from generator import DataGenerator, DataGeneratorWithCoords, DataGeneratorWithCoordsRandom

import matplotlib.pyplot as plt
sys.path.append('../../../dataset/dataset/patches_extract_script/')
from dataSource import DataSource, SARSource, OpticalSource, Dataset, LEM, LEM2, CampoVerde, OpticalSourceWithClouds, Humidity
from model_input_mode import MIMFixed, MIMVarLabel, MIMVarSeqLabel, MIMVarLabel_PaddedSeq, MIMFixedLabelAllLabels, MIMFixed_PaddedSeq
from parameters.parameters_reader import ParamsTrain

from icecream import ic
from monitor import Monitor, MonitorNPY, MonitorGenerator, MonitorNPYAndGenerator
import natsort
from model import NetModel, ModelFit, ModelLoadGeneratorWithCoords
from dataset import Dataset, DatasetWithCoords

from patch_extractor import PatchExtractor
ic.configureOutput(includeContext=False)
np.random.seed(2021)
tf.random.set_seed(2021)

from main import TrainTest

In [18]:
!nvidia-smi

Fri Aug 20 22:44:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Set parameters

Parameters can be modified in /content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src/parameters/parameters_reader.py

In [23]:
from pathlib import Path

paramsTrain = ParamsTrain('parameters/')
paramsTrain.mim = MIMFixed_PaddedSeq()

paramsTrain.getFullIms = False
paramsTrain.coordsExtract = False
paramsTrain.train = True

paramsTrain.train_overlap_percentage = 0
paramsTrain.trainGeneratorRandom = False
paramsTrain.patch_len = 32
paramsTrain.stride = int(paramsTrain.patch_len - paramsTrain.patch_len * paramsTrain.train_overlap_percentage)
paramsTrain.patch_step_train = paramsTrain.stride
paramsTrain.patch_step_test = paramsTrain.patch_len # to do: paramsTrain.getCalculatedParams() does these calculations

paramsTrain.dataset = 'cv'
paramsTrain.seq_date = 'jun'
paramsTrain.path = Path("../../../dataset/dataset/") / (paramsTrain.dataset + "_data")

paramsTrain.test_overlap_percentage = 0

paramsTrain.dataSource = SARSource()
paramsTrain.openSetMethod = None
trainTest = TrainTest(paramsTrain)

patchExtractor = PatchExtractor(paramsTrain, trainTest.ds)	

[@debug] self.seq_date: 'mar'


self.known_classes [0, 1, 10, 12]


[@debug] self.stride: 32


['parameters_reader.py', 'save_nonaugmented_train_patches_unknownclasses.json', 'save_nonaugmented_train_patches.json', 'cv', 'twokkc_parameters_closedset_groupclasses.json', 'twokkc_parameters_openset.json', 'twokkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches_lessclass8.json', 'allkkc_save_nonaugmented_train_patches.json', 'params_batchprocessing.py', 'parameters_openset_lessclass8.json', '__init__.py', 'params_reconstruct.py', 'no_mode.json', 'allkkc_parameters_openset.json', 'parameters_openset.json', 'parameters_closedset_groupclasses.json', 'parameters_openset_specifyunknownclasses.json', 'parameters_closedset_groupclasses_lessclass8.json']
[@debug] self.seq_mode = fixed
[@debug] self.mim = <model_input_mode.MIMFixed_PaddedSeq object at 0x7f7b2037aa90>
[@debug] self.ds = <dataSource.CampoVerde object at 0x7f7b2037aa90>
20151029
20151110
20151122
20151204
20151216
20160121
20160214
20160309
20160321
20160508
20160520
20160613
dotys_sin_cos.shape (12, 2)


[@debug] self.dataSource: <dataSource.SARSource object at 0x7f7b2037a7d0>
[@debug] self.conf['path']/self.label_folder/"/": PosixPath('/')
[@debug] self.conf["in_npy_path"]: PosixPath('../../../dataset/dataset/cv_data/in_sar')
[@debug] self.conf["train"]["mask"]["dir"]: PosixPath('../../../dataset/dataset/cv_data/TrainTestMask.tif')
[@debug] os.getcwd(): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'


## Download or load sequence of images


In [24]:
if paramsTrain.getFullIms == True:
  patchExtractor.getFullIms()	
else:
  patchExtractor.fullImsLoad()


## Extract coords of image patches

In [25]:

if paramsTrain.coordsExtract == True:
  patchExtractor.extract()

del patchExtractor



## Train


In [26]:
	trainTest.setData()


	trainTest.preprocess(paramsTrain.model_name_id) # move into if

	trainTest.setModel()

	if paramsTrain.train == True:
		trainTest.train()
	else:
		trainTest.modelLoad(paramsTrain.model_name_id)

Initializing object...
12 2
[@debug] self.channel_n = 2


[@debug] self.class_n: 10


[@debug] self.t_len = 12
Initializing Dataset instance


[@debug] self.data.class_n: 10
[@debug] self.model_name: '../results/convlstm_results/model/lm/model_best_UUnetConvLSTM_mar_lm_nomode.h5'
[@debug] os.path.dirname(os.path.abspath(__file__)): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'
[@debug] os.getcwd(): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'
[@debug] self.patches['train']['coords'].shape: (4983, 2)
[@debug] np.unique(self.full_label_train, return_counts=True): (array([ 0,  2,  3,  4,  6,  7,  8,  9, 10, 11], dtype=uint8),
                                                                array([64671486,   710232,  1374508,    31550,   136800,   212701,
                                                                         71804,   617076,      699,    66684]))
[@debug] np.unique(self.full_label_test, return_counts=True): (array([ 0,  2,  3,  4,  6,  7,  8,  9, 10, 11], dtype=uint8),
                                                             

[@debug] all_classes = [ 1  2  3  5  6  7  8  9 10]
[@debug] self.paramsTrain.known_classes = [1, 2, 6, 8]
[@debug] self.unknown_classes = [ 3  5  7  9 10]


...

=== SELECT VALIDATION SET FROM TRAIN SET
[@debug] self.data.patches['val']['label'].shape = (1, 1)
[@debug] self.data.patches['train']['coords'].shape = (4983, 2)
Initializing object...
12 2
[@debug] self.channel_n = 2
[@debug] self.t_len = 12
Initializing Model instance
[@debug] self.mp = {'dense': {'recurrent_filters': 128, 'nb_dense_block': 2, 'growth_rate': 64, 'nb_layers_per_block': 1}, 'unet': {'recurrent_filters': 128, 'filter_size': 16}, 'atrous': {'recurrent_filters': 128, 'filter_size': 16, 'dilation_rate_mode': 'auto', 'dilation_rates': [1, 2, 4, 8]}}
[@debug] self.stop_epoch = 400


[@debug] self.model.name: PosixPath('../results/convlstm_results/model/lm/../results/convlstm_results/model/lm/model_best_UUnetConvLSTM_mar_lm_nomode.h5')
[@debug] self.model.class_n: 4
[@debug] self.data.class_n: 5


[@debug] self.data.class_n = 5
[@debug] self.t_len = 12
[@debug] self.model_t_len = 12
[@debug] K.int_shape(x) = (None, 12, 8, 8, 64)
[@debug] K.int_shape(res2) = (None, 8, 8, 64)
[@debug] K.int_shape(p3) = (None, 8, 8, 64)
[@debug] K.int_shape(d3) = (None, 8, 8, 64)
[@debug] K.int_shape(x) = (None, 12, 16, 16, 32)
[@debug] K.int_shape(res2) = (None, 16, 16, 32)
[@debug] K.int_shape(p2) = (None, 16, 16, 32)
[@debug] K.int_shape(d2) = (None, 16, 16, 32)
[@debug] K.int_shape(x) = (None, 12, 32, 32, 16)
[@debug] K.int_shape(res2) = (None, 32, 32, 16)
[@debug] K.int_shape(p1) = (None, 32, 32, 16)
[@debug] K.int_shape(d1) = (None, 32, 32, 16)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 12, 32, 32,  0                                            
________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
[@debug] data.patches['train']['coords'].shape: (4983, 2)
[@debug] data.t_len: 12
[@debug] data.full_ims_train.shape: (12, 8492, 7995, 2)
[@debug] self.model_t_len: 12
[@debug] im.shape: (12, 8492, 7995, 2)
[@debug] data.full_ims_train.shape: (12, 8492, 7995, 2)
[@debug] self.class_n: 4
[@debug] data.patches['train']['coords'].shape: (4983, 2)
[@debug] data.patches['train']['coords'][0:16]: array([[7568,  656],
                                                       [7600,  656],
                                                       [7536,  688],
                                                       [7568,  688],
                                                       [7600,  688],
                                                       [7632,  688],
                   

KeyError: ignored

## Evaluate


In [ ]:
trainTest.evaluate()